In [2]:
# import needed
import pandas as pd # data processing
import pandas_profiling as pp
import numpy as np # linear algebra
import pprint #indet json 
import requests #make http requests
from qwikidata.sparql  import return_sparql_query_results #return sparql results

In [ ]:
df_netflix = pd.read_csv('originalDataset/netflix_titles.csv')
df_amazon = pd.read_csv('originalDataset/amazon_prime_titles.csv')
print(len(df_netflix))
print(len(df_amazon))

: 

: 

In [6]:
df_netflix.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...


In [7]:
df_amazon.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...


In [8]:
df_netflix["date_added"] = pd.to_datetime(df_netflix['date_added']) #turn dates into dates


""" df_netflix['month_added']=df_netflix['date_added'].dt.month
df_netflix['month_name_added']=df_netflix['date_added'].dt.month_name() #add separate column 4 year date month(?)
df_netflix['year_added'] = df_netflix['date_added'].dt.year
 """
df_netflix.head(3)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...


In [14]:
netflix = df_netflix.query("type == 'Movie'") #keep only movies
netflix.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
netflix.head(3)

NameError: name 'df_netflix' is not defined

In [13]:

#QUERY!!!!!!!!!!!!!

'''SELECT ?film ?film_label ?director ?director_label ?dir_gen ?dir_gen_label ?distributor ?distributor_label ?review_score
WHERE
{
  ?film wdt:P31 wd:Q11424 .
  ?film rdfs:label "Zubaan"@en .
  ?film rdfs:label ?film_label .
  FILTER(lang(?film_label) = 'en')
  ?film wdt:P57 ?director . 
  ?director rdfs:label ?director_label .    
  FILTER(lang(?director_label) = 'en')
  ?director wdt:P21 ?dir_gen . 
  ?dir_gen rdfs:label ?dir_gen_label .
  FILTER(lang(?dir_gen_label) = 'en')
  ?film wdt:P750 ?distributor . 
  ?distributor rdfs:label ?distributor_label .
  FILTER(lang(?distributor_label) = 'en')
  ?film wdt:P444 ?review_score .
}'''

'SELECT ?film ?film_label ?director ?director_label ?dir_gen ?dir_gen_label ?distributor ?distributor_label ?review_score\nWHERE\n{\n  ?film wdt:P31 wd:Q11424 .\n  ?film rdfs:label "Zubaan"@en .\n  ?film rdfs:label ?film_label .\n  FILTER(lang(?film_label) = \'en\')\n  ?film wdt:P57 ?director . \n  ?director rdfs:label ?director_label .    \n  FILTER(lang(?director_label) = \'en\')\n  ?director wdt:P21 ?dir_gen . \n  ?dir_gen rdfs:label ?dir_gen_label .\n  FILTER(lang(?dir_gen_label) = \'en\')\n  ?film wdt:P750 ?distributor . \n  ?distributor rdfs:label ?distributor_label .\n  FILTER(lang(?distributor_label) = \'en\')\n  ?film wdt:P444 ?review_score .\n}'

In [14]:
p = pprint.PrettyPrinter(indent=1)
# funzia piu o mno come una ricerca su wikidata
API_WD = "https://www.wikidata.org/w/api.php"
params = {
    'action': 'wbsearchentities',# <------------ web search from wikidata
    'format': 'json',
    'language': 'en',
    'search': 'Dick Johnson Is Dead' # the query string
}

# query wd API    
r = requests.get(API_WD, params = params).json() 
p.pprint(r)

{'search': [{'concepturi': 'http://www.wikidata.org/entity/Q84366622',
             'description': '2020 documentary film by Kirsten Johnson',
             'display': {'description': {'language': 'en',
                                         'value': '2020 documentary film by '
                                                  'Kirsten Johnson'},
                         'label': {'language': 'en',
                                   'value': 'Dick Johnson Is Dead'}},
             'id': 'Q84366622',
             'label': 'Dick Johnson Is Dead',
             'match': {'language': 'en',
                       'text': 'Dick Johnson Is Dead',
                       'type': 'label'},
             'pageid': 83667587,
             'repository': 'wikidata',
             'title': 'Q84366622',
             'url': '//www.wikidata.org/wiki/Q84366622'}],
 'searchinfo': {'search': 'Dick Johnson Is Dead'},
 'success': 1}


In [15]:
from SPARQLWrapper import SPARQLWrapper, JSON #questo serve a vedere la struttura delle risposte
import ssl

p = pprint.PrettyPrinter(indent=1)
ssl._create_default_https_context = ssl._create_unverified_context #????? ceracare poi (lezioni daquino)

# get the endpoint API
wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql" #<---------- Non è l'endpoint, stiamo fetchando della documentazione di qualche tipo

# prepare the query : 10 random triples
my_SPARQL_query = """
SELECT *
WHERE {?s ?p ?o}
LIMIT 10
"""

# set the endpoint 
sparql_wd = SPARQLWrapper(wikidata_endpoint)
# set the query
sparql_wd.setQuery(my_SPARQL_query)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()

p.pprint(results)

{'head': {'vars': ['s', 'p', 'o']},
 'results': {'bindings': [{'o': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
                                 'type': 'literal',
                                 'value': '2021-09-24T23:11:29Z'},
                           'p': {'type': 'uri',
                                 'value': 'http://schema.org/dateModified'},
                           's': {'type': 'uri',
                                 'value': 'http://wikiba.se/ontology#Dump'}},
                          {'o': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
                                 'type': 'literal',
                                 'value': '2021-09-24T23:11:31Z'},
                           'p': {'type': 'uri',
                                 'value': 'http://schema.org/dateModified'},
                           's': {'type': 'uri',
                                 'value': 'http://wikiba.se/ontology#Dump'}},
                          {'o': {'datatype':

In [3]:
import time
for x in range(6):
  if x == 3:
    time.sleep(5)
    print(x)
  else:
    print(x)


0
1
2
3
4
5


In [17]:
title = list(netflix['title']) #printa una lista di titoli da netflix
#print(title[0:100], title[100:200])
#print(title[0:100], title[100:200])
print(len(title))


6131


In [22]:
#print(film)
ciccio_prova = title[0:20] 
print(ciccio_prova)


['Dick Johnson Is Dead', 'My Little Pony: A New Generation', 'Sankofa', 'The Starling', 'Je Suis Karl', 'Confessions of an Invisible Girl', "Europe's Most Dangerous Man: Otto Skorzeny in Spain", 'Intrusion', 'Avvai Shanmughi', 'Go! Go! Cory Carson: Chrissy Takes the Wheel', 'Jeans', 'Minsara Kanavu', 'Grown Ups', 'Dark Skies', 'Paranoia', 'Ankahi Kahaniya', 'The Father Who Moves Mountains', 'The Stronghold', 'Birth of the Dragon', 'Jaws']


In [ ]:
""" print("film1 should be []")
print(film) """


p = pprint.PrettyPrinter(indent=1)

def wikidata_reconciliation(query):

    oneundred_ttls = [ciccio_prova[x:x + 100] for x in range(0, len(ciccio_prova), 100)]
    for i in range(0, len(oneundred_ttls)):
        time.sleep(25)
        

        ssl._create_default_https_context = ssl._create_unverified_context
    
    
    # get the endpoint API
        wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
        """ print("query = title[0:2]")
        print(query)
        print("film should be []")
        print(film) """

        for x in oneundred_ttls[i]:
            print(x)
            my_SPARQL_query = """
            SELECT ?film ?film_label ?director ?director_label ?dir_gen ?dir_gen_label ?distributor ?distributor_label ?review_score
            WHERE
            {
            ?film wdt:P31 wd:Q11424 .
            ?film rdfs:label """+'"'+ x +'"' +"""@en .
            ?film rdfs:label ?film_label .
            FILTER(lang(?film_label) = 'en')
            ?film wdt:P57 ?director . 
            ?director rdfs:label ?director_label .    
            FILTER(lang(?director_label) = 'en')
            ?director wdt:P21 ?dir_gen . 
            ?dir_gen rdfs:label ?dir_gen_label .
            FILTER(lang(?dir_gen_label) = 'en')
            ?film wdt:P750 ?distributor . 
            ?distributor rdfs:label ?distributor_label .
            FILTER(lang(?distributor_label) = 'en')
            ?film wdt:P444 ?review_score .
            }
            """
            # set the endpoint 
            sparql_wd = SPARQLWrapper(wikidata_endpoint)
            # set the query
            sparql_wd.setQuery(my_SPARQL_query)
            # set the returned format
            sparql_wd.setReturnFormat(JSON)
            # get the results
            
            results = sparql_wd.query().convert()
            response = requests.head(wikidata_endpoint)
            """if response.status_code == 429:
                print("riproviamo")
                time.sleep(int(response.headers["Retry-After"]))"""
            
            if results['results']['bindings'] == []:
                print(""+x+" not found")
            else:
                film.append(results['results']['bindings'][0]['film_label']['value'])
                director.append(results['results']['bindings'][0]['director_label']['value'])
                gender.append(results['results']['bindings'][0]['dir_gen_label']['value'])
                distributor.append(results['results']['bindings'][0]['distributor_label']['value'])
                    
            
    print(len(film))



wikidata_reconciliation(ciccio_prova)

#wikidata_reconciliation(title[100:200])


Dick Johnson Is Dead
My Little Pony: A New Generation
My Little Pony: A New Generation not found
Sankofa
Sankofa not found
The Starling
Je Suis Karl
Je Suis Karl not found
Confessions of an Invisible Girl
Confessions of an Invisible Girl not found
Europe's Most Dangerous Man: Otto Skorzeny in Spain
Europe's Most Dangerous Man: Otto Skorzeny in Spain not found
Intrusion
Intrusion not found
Avvai Shanmughi
Avvai Shanmughi not found
Go! Go! Cory Carson: Chrissy Takes the Wheel
Go! Go! Cory Carson: Chrissy Takes the Wheel not found
Jeans
Jeans not found
Minsara Kanavu
Minsara Kanavu not found
Grown Ups
Dark Skies
Paranoia
Ankahi Kahaniya
Ankahi Kahaniya not found
The Father Who Moves Mountains
The Father Who Moves Mountains not found
The Stronghold
The Stronghold not found
Birth of the Dragon
Jaws
Jaws 2
Jaws 3
Jaws 3 not found
Jaws: The Revenge
My Heroes Were Cowboys
My Heroes Were Cowboys not found
Safe House
Training Day
InuYasha the Movie 2: The Castle Beyond the Looking Glass
InuYasha

HTTPError: HTTP Error 429: Too Many Requests

# New

In [4]:
df_newdata = pd.read_csv('new_data.csv')
df_newdata = df_newdata.loc[:, ~df_newdata.columns.str.contains('^Unnamed')]
df_newdata.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df_newdata['title'] = df_newdata['title'].replace({'"':''}, regex=True)

df_newdata.head(2)
#print(len(df_newdata))

,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
0,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",No Data,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,1,0,1000
1,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",No Data,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...,1,0,1000


In [5]:
movie_to_find = df_newdata.query("type == 'Movie'") #keep only movies
serie_to_find = df_newdata.query("type == 'TV Show'")


movie_to_find.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
movie_to_find.head(1)

,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
0,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",No Data,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,1,0,1000


In [6]:
print(len(df_newdata))
print(len(movie_to_find))
print(len(serie_to_find))
print(12214 -12036)

12214
12036
178
178


In [7]:
movie_title = list(movie_to_find['title'])


print(movie_title.index("Gabriel Fluffy Iglesias: One Show Fits All"))
#print(movie_title[2399:])
print(len(movie_title))
print(movie_title[11876:11877])
print(movie_title.index("Zinzana"))


2399
12036
['Cover-Up']
5513


In [ ]:
# old query
"""
            SELECT ?film ?film_label ?director ?director_label ?dir_gen ?dir_gen_label ?distributor ?distributor_label ?review_score
            WHERE
            {
            ?film wdt:P31 wd:Q11424 .
            ?film rdfs:label """+'"'+ x +'"' +"""@en .
            ?film rdfs:label ?film_label .
            FILTER(lang(?film_label) = 'en')
            OPTIONAL {?film wdt:P57 ?director . 
            ?director rdfs:label ?director_label .    
            FILTER(lang(?director_label) = 'en')}
            OPTIONAL {?director wdt:P21 ?dir_gen . 
            ?dir_gen rdfs:label ?dir_gen_label .
            FILTER(lang(?dir_gen_label) = 'en')}
             OPTIONAL {?film wdt:P750 ?distributor . 
            ?distributor rdfs:label ?distributor_label .
            FILTER(lang(?distributor_label) = 'en')}
            }
"""

In [8]:
from SPARQLWrapper import SPARQLWrapper, JSON #questo serve a vedere la struttura delle risposte
import ssl

p = pprint.PrettyPrinter(indent=1)

In [ ]:
# "In Your Hands" == error 500

In [16]:
distributor = []
imdbID = []
rottenscore = []


In [17]:
film = [] #facciamo un po' di liste che ci divertiamo
director = []
gender = []


In [9]:
import time
not_found = []

In [18]:

p = pprint.PrettyPrinter(indent=1)
import urllib.error
from xml.etree.ElementPath import xpath_tokenizer_re

def wikidata_reconciliation(query):

    
    # get the endpoint API
    wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
        

    for x in query:
        
        try:

            print(x)
            my_SPARQL_query = """
            SELECT ?film ?film_label ?director ?director_label ?dir_gen ?dir_gen_label 
            WHERE
            {
            ?film wdt:P31 wd:Q11424 .
            ?film rdfs:label """+'"'+ x +'"' +"""@en .
            ?film rdfs:label ?film_label .
            FILTER(lang(?film_label) = 'en')
            OPTIONAL {?film wdt:P57 ?director . 
            ?director rdfs:label ?director_label .    
            FILTER(lang(?director_label) = 'en')
            OPTIONAL {?director wdt:P21 ?dir_gen . 
            ?dir_gen rdfs:label ?dir_gen_label .
            FILTER(lang(?dir_gen_label) = 'en')}}
            
            }
            """
            # set the endpoint 
            sparql_wd = SPARQLWrapper(wikidata_endpoint)
            # set the query
            sparql_wd.setQuery(my_SPARQL_query)
            # set the returned format
            sparql_wd.setReturnFormat(JSON)
            # get the results
            
            results = sparql_wd.query().convert()

            if results['results']['bindings'] == []:
                not_found.append(""+x+"")
                print(""+x+" not found")
                
            else:
                film.append(results['results']['bindings'][0]['film_label']['value'])
                if "director_label" in results['results']['bindings'][0]:
                    director.append(results['results']['bindings'][0]['director_label']['value'])
                else:
                    director.append("no_data")
                if "dir_gen_label" in results['results']['bindings'][0]:
                    gender.append(results['results']['bindings'][0]['dir_gen_label']['value'])
                else:
                    gender.append("no_data")
                


        except urllib.error.HTTPError as e:
            print("!!!!!HELP!!!!!")
            print(e.headers)
            print(e.headers["retry-after"])
            time.sleep((int(e.headers["retry-after"])) + 1)
            error_title = query.index(x)
            print(x)
            print(error_title)
            print(query[error_title])
            wikidata_reconciliation(query[error_title:])
            
   



wikidata_reconciliation(movie_title[2637:])

Anjaan
Bathinda Express
Bathinda Express not found
Delhi 6
Delhi 6 not found
Dev.D
Dil Vil Pyaar Vyaar
Dushman
Ek Main Aur Ekk Tu
Freedom at Midnight
Freedom at Midnight not found
Haani
Happy Go Lucky
Harishchandrachi Factory
Jagga Jasoos
Jatt James Bond
Jodhaa Akbar
Judge Singh LLB
Kaminey
Khido Khundi
Khoobsurat
Kisaan
Kismat Konnection
Kurbaan
Lock


KeyboardInterrupt: 

In [58]:
#print(not_found)

['My Little Pony: A New Generation', 'Je Suis Karl']


In [10]:
from http.client import IncompleteRead

In [23]:


import urllib.error
from xml.etree.ElementPath import xpath_tokenizer_re

def wikidata_reconciliation1(query):

    
    # get the endpoint API
    wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
        

    for x in query:

        try:

            
            my_SPARQL_query = """
            SELECT ?film ?film_label ?distributor ?distributor_label ?imdbID ?rottenscore
            WHERE
            {
            ?film wdt:P31 wd:Q11424 .
            ?film rdfs:label """+'"'+ x +'"' +"""@en .
            ?film rdfs:label ?film_label .
            FILTER(lang(?film_label) = 'en')
            OPTIONAL {?film wdt:P750 ?distributor . 
            ?distributor rdfs:label ?distributor_label .
            FILTER(lang(?distributor_label) = 'en')}
            OPTIONAL {?film wdt:P345 ?imdbID.}
            OPTIONAL {?film wdt:P444 ?rottenscore.}
            }
            """
            # set the endpoint 
            sparql_wd = SPARQLWrapper(wikidata_endpoint)
            # set the query
            sparql_wd.setQuery(my_SPARQL_query)
            # set the returned format
            sparql_wd.setReturnFormat(JSON)
            # get the results
            
            results = sparql_wd.query().convert()

            if results['results']['bindings'] == []:
                print(""+x+" not found")
            else:
                film.append(results['results']['bindings'][0]['film_label']['value'])
                if "distributor_label" in results['results']['bindings'][0]:
                    distributor.append(results['results']['bindings'][0]['distributor_label']['value'])
                else:
                    distributor.append("no_data")
                if "imdbID" in results['results']['bindings'][0]:
                    imdbID.append(results['results']['bindings'][0]['imdbID']['value'])
                else:
                    imdbID.append("no_data")
                if "rottenscore" in results['results']['bindings'][0]:
                    rottenscore.append(results['results']['bindings'][0]['rottenscore']['value'])
                else:
                    rottenscore.append("no_data")



        except urllib.error.HTTPError as e:

            print(e.headers["retry-after"])
            time.sleep((int(e.headers["retry-after"])) + 1)
            error_title = query.index(x)
            wikidata_reconciliation1(query[error_title:])
        except IncompleteRead:
            # Oh well, reconnect and keep trucking
            continue
            




wikidata_reconciliation1(movie_title[8089:9000])

Asin_tombstone_test_target not found
You Belong To Me not found
Tom Clancy's Without Remorse not found
The Man From Kathmandu not found
Solar Impact not found
Silence (Tamil) not found
Seduced not found
Scotland, PA not found
Paranormal Investigation Agency not found
Night of a Lone Owl not found
Mike Case in: The Big Kiss Off not found
Maikol Yordan 2: The faraway cure not found
Ghost Box not found
Edgar Allan Poe's The Oval Portrait not found
Dominick Dunne: After The Party not found
Capturing the Light not found
Bacterium not found
100M Criminal Conviction not found
Younger & Younger not found
The Struggle 2: The Dilemma not found
The Manchurian Candidate (2004) not found
The Luck Of Ginger Coffey not found
The House of Screaming Death not found
The House Across The Bay not found
The Bone Garden not found
Sweet Mahogany 2: Pure Indulgence not found
Sugarbaby not found
Ruin Agency not found
Rogue Cell 2: Shadow Warrior not found
Rim Of The Canyon not found
Riders Of The Whistling Pin

In [11]:
print(len(film))

print(len(director))
print(len(gender))
print(len(distributor))
print(len(imdbID))
print(imdbID[-1])
print(film[-1])
print(len(rottenscore))

NameError: name 'film' is not defined

In [1]:
dict = {"title": film, "director": director,
        "director gender": gender}

lists_df = pd.DataFrame(dict)

lists_df.head(5)

NameError: name 'film' is not defined

In [25]:
dict1 = {"title": film, "distributor": distributor, "id": imdbID, "rating score": rottenscore}
lists1_df = pd.DataFrame(dict1)

lists1_df.head(5)

,title,distributor,id,rating score
0,Slappy and the Stinkers,TriStar Pictures,tt0120213,3.3/10
1,Sky Racket,Sam Katzman,tt0140551,no_data
2,Sitting on the Moon,Republic Pictures,tt0028260,no_data
3,Sita,no_data,tt9665402,no_data
4,Sing Sing Nights,no_data,tt0027003,no_data


In [76]:
lists_df.to_csv('savelist.csv')


In [26]:
lists1_df.to_csv('6000-9000.csv')

In [24]:
print(film)


['Dick Johnson Is Dead', 'The Starling', 'Grown Ups', 'Dark Skies', 'Paranoia', 'Birth of the Dragon', 'Jaws']


# Clean New Dataset

In [12]:

#open datasets
df_movie_title = pd.read_csv('savelist.csv')
#df_movie_id = pd.read_csv('savelist2.csv')
df_serie_title = pd.read_csv('series_director.csv')
df_serie_id = pd.read_csv('series.csv')


In [ ]:
#open the tree datasets that contain rottenrate ids distributer

In [13]:

df_first = pd.read_csv('0-6000.csv')
df_first = df_first.loc[:, ~df_first.columns.str.contains('^Unnamed')]
df_first.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df_first.head(5)
df_first.info()
print(len(df_first))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9375 entries, 0 to 9374
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         9375 non-null   object
 1   distributor   9375 non-null   object
 2   id            9375 non-null   object
 3   rating score  9375 non-null   object
dtypes: object(4)
memory usage: 293.1+ KB
9375


In [14]:

df_last = pd.read_csv('9000_end.csv')
df_last = df_last.loc[:, ~df_last.columns.str.contains('^Unnamed')]
df_last.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df_last.head(5)

df_last.info()
print(len(df_last))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3511 entries, 0 to 3510
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         3511 non-null   object
 1   distributor   3511 non-null   object
 2   id            3511 non-null   object
 3   rating score  3511 non-null   object
dtypes: object(4)
memory usage: 109.8+ KB
3511


In [15]:
df_but_last = pd.read_csv('6000-9000.csv')
df_but_last = df_but_last.loc[:, ~df_but_last.columns.str.contains('^Unnamed')]
df_but_last.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df_but_last.head(5)
df_but_last.info()
print(len(df_but_last))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1331 entries, 0 to 1330
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         1331 non-null   object
 1   distributor   1331 non-null   object
 2   id            1331 non-null   object
 3   rating score  1331 non-null   object
dtypes: object(4)
memory usage: 41.7+ KB
1331


In [16]:
#drop duplicates
df_but_last_copy = df_but_last.drop_duplicates(keep='first')
df_last_copy = df_last.drop_duplicates(keep='first')
df_first_copy = df_first.drop_duplicates(keep='first')

In [17]:
#make a single dataset -> df_movie_id

df_movie_id = pd.concat([df_first_copy, df_but_last_copy, df_last_copy],axis=0, join="outer", sort=False)
df_movie_id.head(5)
#print(len(df_first_copy))
#print(len(df_but_last_copy))
#print(len(df_last_copy))
#print(len(df_first_copy)+len(df_but_last_copy)+len(df_last_copy))
#print(len(df_movie_id))

#print(df_movie_id.info())


,title,distributor,id,rating score
0,Sankofa,no_data,tt0108041,8.2/10
1,The Starling,Netflix,tt5164438,4.2/10
2,Confessions of an Invisible Girl,Netflix,tt15204288,no_data
3,Intrusion,no_data,tt5563324,4.5/10
4,Avvai Shanmughi,no_data,tt0139872,no_data


In [148]:
df_movie_id.to_csv('prova.csv')

In [149]:
df_movie_id.query("title == 'Harry Brown'")


,title,distributor,id,rating score
1405,Harry Brown,Lionsgate,tt1289406,63%


# oi

In [155]:


titles = df_movie_id['title'].tolist()
print(titles[-1])

7118
The Trench


In [207]:

#sistema gli index
df_movie_title = df_movie_title.loc[:, ~df_movie_title.columns.str.contains('^Unnamed')]
df_movie_title.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')

#df_movie_id = df_movie_id.loc[:, ~df_movie_id.columns.str.contains('^Unnamed')] l'index è gia a posto
#df_movie_id.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')

df_serie_title = df_serie_title.loc[:, ~df_serie_title.columns.str.contains('^Unnamed')]
df_serie_title.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')

df_serie_id = df_serie_id.loc[:, ~df_serie_id.columns.str.contains('^Unnamed')]
df_serie_id.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')

In [211]:
#problema #1 dobbiamo essere sicuri che gli autori dei film scaricati da wikidata matchino quelli che abbiamo già(quindi non avere film con stesso titolo ma diverso autore)

df_movie_title.head(7)
#df_movie_title.info()
#print(len(df_movie_title))

,title,director,director gender
0,Sankofa,Haile Gerima,male
1,The Starling,Theodore Melfi,male
2,Confessions of an Invisible Girl,no_data,no_data
3,Intrusion,no_data,no_data
4,Avvai Shanmughi,K. S. Ravikumar,male
5,Jeans,Nicolette Krebitz,female
6,Minsara Kanavu,Rajiv Menon,male


In [225]:
match_titles = df_movie_title['title'].unique()
print(len(match_titles))
match_titles1 = df_movie_id['title'].unique()
print(len(match_titles1))

6677
6677


In [206]:
ids = df_movie_id['id'].tolist()
print(ids)

['tt0108041', 'tt5164438', 'tt15204288', 'tt5563324', 'tt0139872', 'tt0279885', 'tt0282003', 'tt1375670', 'tt2387433', 'tt1413495', 'tt15250998', 'no_data', 'tt5238240', 'tt2720826', 'tt0073195', 'tt0077766', 'tt0093300', 'tt1599348', 'tt0139654', 'tt10521144', 'tt10322274', 'tt8329618', 'no_data', 'tt7737528', 'tt0468536', 'tt15095938', 'tt6127004', 'tt5691024', 'tt10168094', 'tt6167894', 'tt8009744', 'tt11742798', 'tt0109134', 'tt0810784', 'tt4088588', 'tt4699202', 'tt1118045', 'tt0199626', 'tt10930586', 'tt0356470', 'tt0350261', 'tt1823672', 'tt0109444', 'tt0106582', 'tt0159365', 'tt0231402', 'tt0989757', 'tt0097216', 'tt4382872', 'tt0463998', 'tt1133985', 'tt0099800', 'tt0102065', 'tt0110064', 'tt0131436', 'tt0099834', 'tt0439630', 'tt4841640', 'tt0091369', 'tt0892318', 'tt4006302', 'tt0337592', 'tt1602479', 'tt0116996', 'tt5301662', 'tt0270980', 'tt0132347', 'tt0087843', 'tt0409182', 'tt0120014', 'tt1230168', 'tt0332379', 'tt5073746', 'tt0314353', 'tt0080453', 'tt0091129', 'tt0054

In [163]:
# droppiamo i duplicati per essere sicuri che nel merge non sino saltati fuori dei doppioni
df_movie_title_copy = df_movie_title.drop_duplicates(keep='first')
print(len(df_movie_title_copy))

df_movie_id_copy = df_movie_id.drop_duplicates(keep='first')
len(df_movie_id_copy)


6688


7034

In [212]:
#controlliamo i film che hanno stesso titolo ma diverso autore
pd.concat(g for _, g in df_movie_title_copy.groupby("title") if len(g) > 1)

,title,director,director gender
2410,Abhinetri,Subodh Mukherjee,male
5692,Abhinetri,Satish Pradhan,male
322,Bliss,Abdullah Oğuz,male
6057,Bliss,Doris Dörrie,female
924,Boss,R. Raghuraj,male
4464,Boss,V. N. Aditya,male
6774,Jack,Francis Ford Coppola,male
6827,Jack,Elisabeth Scharang,female
1594,Lionheart,Genevieve Nnaji,female
5071,Lionheart,Sheldon Lettich,male


In [226]:
df_movie_title_copy.query('title == "Abhinetri"')

,title,director,director gender
2410,Abhinetri,Subodh Mukherjee,male
5692,Abhinetri,Satish Pradhan,male


In [24]:
df_newdata.head(2) #riprendiamo il nostro dataset netflix/amazon

,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
0,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",No Data,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,1,0,1000
1,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",No Data,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...,1,0,1000


In [25]:
pd.concat(g for _, g in df_newdata.groupby("title") if len(g) > 1) 

,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
3695,Movie,A Family Man,Mark Williams,"Gerard Butler, Gretchen Mol, Alison Brie, Will...","Canada, United States","December 15, 2019",2016,R,110 min,Dramas,A ruthless corporate headhunter battles his ri...,1,0,1000
12130,Movie,A Family Man,Mark Williams,"Gerard Butler, Gretchen Mol, Alison Brie, Will...","United States, Canada",1000,2017,R,110 min,Drama,Dane Jensen (Gerard Butler) is a hard-driven h...,0,1,1000
2461,Movie,A Second Chance,Cathy Garcia-Molina,"John Lloyd Cruz, Bea Alonzo, Dimples Romana, J...",Philippines,"February 27, 2019",2015,TV-MA,130 min,"Dramas, International Movies, Romantic Movies","Following their storybook wedding, Popoy and B...",1,0,1000
7324,Movie,A Second Chance,Grayson Stroud,"Chico Benymon, Jennifer Freeman, Malinda Willi...",No Data,1000,2019,13+,94 min,Comedy,When best friends Daniella and Cheyann both st...,0,1,1000
3805,Movie,APEX: The Story of the Hypercar,"J.F. Musial, Josh Vietze",Zachary Levi,United States,"August 1, 2016",2016,NR,86 min,Documentaries,A visionary carmaker introduces his fuel-effic...,1,1,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12066,Movie,What Lies Beneath,Menno Meyjes,"Monic Hendrickx, Fedja van Huêt, Daan Schuurma...",No Data,1000,2016,13+,95 min,Suspense,"After many years abroad, a woman's homecoming ...",0,1,1000
5657,Movie,Wolves,Bart Freundlich,"Michael Shannon, Carla Gugino, Taylor John Smi...",United States,"March 29, 2019",2016,R,109 min,"Dramas, Independent Movies, Sports Movies",A promising high school basketball player has ...,1,1,1000
12019,Movie,Wolves,David Hayter,"Jason Momoa, Lucas Till, Stephen McHattie, Mer...",No Data,1000,2014,R,90 min,"Action, Horror, Science Fiction",A high-school student wakes to find he is chan...,0,1,1000
60,Movie,Worth,Sara Colangelo,"Michael Keaton, Stanley Tucci, Amy Ryan, Shuno...",No Data,"September 3, 2021",2021,PG-13,119 min,Dramas,"In the wake of the Sept. 11 attacks, a lawyer ...",1,1,1000


In [26]:
dir_title = []#facciamo touple director/title prendendo i titoli da wikidata e i director del dataset netflix/amazon
unique_titles = df_movie_title_copy['title'].unique()
wikidata_titles = unique_titles.tolist()
print(len(wikidata_titles))

for  i, x in df_newdata['title'].iteritems():
    
    director = df_newdata['director'][i]
    title = x
    if title in wikidata_titles:
        dir_title.append((title, director))
print(len(dir_title)) # aumenta perchè nel dataset netflix/amazon abbiamo diversi film con stesso titolo ma più autori
print(dir_title[0:7])

# df_amazon['title']

6677
6779
[('Sankofa', 'Haile Gerima'), ('The Starling', 'Theodore Melfi'), ('Confessions of an Invisible Girl', 'Bruno Garotti'), ('Intrusion', 'Adam Salky'), ('Avvai Shanmughi', 'K.S. Ravikumar'), ('Jeans', 'S. Shankar'), ('Minsara Kanavu', 'Rajiv Menon')]


In [213]:
df_movie_title_copy.query('title=="Abhinetri"')

,title,director,director gender
2410,Abhinetri,Subodh Mukherjee,male
5692,Abhinetri,Satish Pradhan,male


In [230]:
index_to_drop =[] #ripuliamo il dataframe di wikidata da tutti i film con autori che non matchano
for  index_n, title in df_movie_title_copy['title'].iteritems():
    director = df_movie_title_copy['director'][index_n]
    for tupla in dir_title:
        if tupla[0] == title and tupla[1] != director and director != "no_data" :

            index_to_drop.append(index_n)
    
df_movie_directors = df_movie_title_copy.drop(index_to_drop)
        
        
print(len(index_to_drop))
    

2018


In [215]:
df_movie_directors.head(7)#dataframe con giusti autori

,title,director,director gender
0,Sankofa,Haile Gerima,male
1,The Starling,Theodore Melfi,male
2,Confessions of an Invisible Girl,no_data,no_data
3,Intrusion,no_data,no_data
6,Minsara Kanavu,Rajiv Menon,male
8,Dark Skies,Scott Stewart,male
9,Paranoia,Robert Luketic,male


In [189]:
wikidata_titles_clean_autors = df_movie_directors["title"].tolist()
print(wikidata_titles_clean_autors[0:40]) #facciamo una lista dei titoli con autori che matchano

['Sankofa', 'Intrusion', 'Dark Skies', 'The Father Who Moves Mountains', 'Jaws', 'Jaws 2', 'Jaws: The Revenge', 'Training Day', 'Kate', 'Paradise Hills', 'Angamaly Diaries', 'Bright Star', 'Dhanak', 'Gurgaon', 'Shikara', 'An Unfinished Life', 'Chappie', 'Do the Right Thing', 'Green Lantern', 'In Too Deep', 'Love in a Puff', 'Mystery Men', 'School of Rock', 'Snervous Tyler Oakley', 'The Golden Child', 'The Guns of Navarone', 'Welcome Home Roscoe Jenkins', 'In the Line of Fire', 'C Kkompany', "He's All That", 'Koi Aap Sa', 'Kyaa Kool Hai Hum', 'Kyaa Super Kool Hain Hum', 'Kyo Kii... Main Jhuth Nahin Bolta', 'Ragini MMS 2', 'Shootout at Lokhandwala', 'The Dirty Picture', 'The Old Ways', 'A Faraway Land', 'Like Crazy']


In [ ]:
#problemi futuri
# 1 doppio id per stesso titolo(speriamo non siano troppi controllo a vista)
# 2 cancellati film con stesso autore ma scritto in modo diverso/presenti più autori(forse necessario controllo a vista)

In [237]:

pd.concat(g for _, g in df_movie_directors.groupby("title") if len(g) > 1) 

ValueError: No objects to concatenate

In [227]:
""" for x in dir_title:
    if x[0]=="Abhinetri":
        print(x)
for x in dir_title:
    if x[0]=="Bliss":
        print(x) """
for x in dir_title:
    if x[0]=="Kucch To Hai":
        print(x)
#df_movie_title_copy.query('title=="Avvai Shanmughi"')
#df_movie_title_copy.query('title=="Sankofa"')
df_movie_title_copy.query('title=="Kucch To Hai"')
#df_movie_title_copy.query('title=="The Witcher: Nightmare of the Wolf"')


('Kucch To Hai', 'Anil V. Kumar, Anurag Basu')


,title,director,director gender
87,Kucch To Hai,Anurag Basu,male


In [232]:
df_movie_directors.query('title=="Kucch To Hai"')

,title,director,director gender


In [31]:
#problema #2 abbiamo diversi distributor e rating per stesso titolo/id
pd.concat(g for _, g in df_movie_id_copy.groupby("title") if len(g) > 1)

,title,distributor,id,rating score
1401,10 Things I Hate About You,Disney+,tt0147800,69%
1511,10 Things I Hate About You,Netflix,tt0147800,69%
1258,10 to Midnight,The Cannon Group,tt0085121,40%
2612,10 to Midnight,Netflix,tt0085121,40%
772,A Chinese Ghost Story,no_data,tt1861421,no_data
...,...,...,...,...
6998,Zandalee,Netflix,tt0101004,4/10
3784,Zoe,Netflix,tt6010628,4.3/10
5546,Zoe,Amazon Studios,tt6010628,4.3/10
3780,Zombieland,InterCom,tt1156398,7.4/10


In [177]:
df_movie_id_copy.head(7)
df_movie_id_copy.query('title=="10 Things I Hate About You"')

,title,distributor,id,rating score
1401,10 Things I Hate About You,Disney+,tt0147800,69%
1511,10 Things I Hate About You,Netflix,tt0147800,69%


In [101]:
#df_movie_id_copy.iloc[3]
distr =[]
for i in df_movie_id_copy["distributor"][0]:
    distrib = df_movie_id_copy["distributor"][0]
    distr.append(i)
    di= tuple(distr)
print(di)

('no_data', 'TriStar Pictures', 'no_data')


In [105]:
df_movie_id_copy.at[1511,'distributor']

1511    Star Cinema
1511        Netflix
Name: distributor, dtype: object

In [178]:
df_movie_id_clean = df_movie_id_copy.groupby(['title', 'id']).agg(tuple).reset_index()
df_movie_id_clean.head(7)  #prima trasformiamo in tuple i valori che hanno in comune la stessa coppia id/title (succede anche se il valore è singolo ma così abbiamo le tuple fatte)

,title,id,distributor,rating score
0,#Alive,tt10620868,"(Netflix,)","(88%,)"
1,#Home,tt10534500,"(Amazon Prime Video,)","(no_data,)"
2,#Roxy,tt5646082,"(iTunes,)","(no_data,)"
3,#realityhigh,tt6119504,"(Netflix,)","(40%,)"
4,'76,tt2385126,"(YouTube,)","(no_data,)"
5,1 Chance 2 Dance,tt2388856,"(Netflix,)","(no_data,)"
6,1 Mile to You,tt2184233,"(Gravitas Ventures,)","(no_data,)"


In [179]:
df_movie_id_clean.query('title=="10 Things I Hate About You"')	

,title,id,distributor,rating score
13,10 Things I Hate About You,tt0147800,"(Disney+, Netflix)","(69%, 69%)"


In [180]:
df_movie_id_clean["distributor"]

0                  (Netflix,)
1       (Amazon Prime Video,)
2                   (iTunes,)
3                  (Netflix,)
4                  (YouTube,)
                ...          
6817               (no_data,)
6818    (Paramount Pictures,)
6819               (Netflix,)
6820               (Netflix,)
6821               (Netflix,)
Name: distributor, Length: 6822, dtype: object

In [181]:
df_movie_id_clean_copy = df_movie_id_clean.copy() #estraiamo i valori singoli dalle tuple e manteniamo un vlore solo se sono duplicati i rotten rates
for n_index, title in df_movie_id_clean_copy['title'].iteritems():
    distributor = df_movie_id_clean_copy["distributor"][n_index]
    rating = df_movie_id_clean_copy["rating score"][n_index]
    
    if len(distributor)== 1:
        df_movie_id_clean_copy.at[n_index,'distributor']=distributor[0]

    if len(rating) > 1:
        
        for i in rating:
            rating_check = set()
            rating_check.add(i)
            
            if len(rating_check) <= 1:
                df_movie_id_clean_copy.at[n_index,'rating score']=rating[0]
                

    if len(rating) == 1:
        
        df_movie_id_clean_copy.at[n_index,'rating score']=rating[0] #puo darsi che tuplare e stuplare i valori non sia bellissimo ma alla fine mi è sembrato più semplice
        
   

In [182]:
df_movie_id_clean_copy

,title,id,distributor,rating score
0,#Alive,tt10620868,Netflix,88%
1,#Home,tt10534500,Amazon Prime Video,no_data
2,#Roxy,tt5646082,iTunes,no_data
3,#realityhigh,tt6119504,Netflix,40%
4,'76,tt2385126,YouTube,no_data
...,...,...,...,...
6817,Zubaan,tt3776484,no_data,no_data
6818,Æon Flux,tt0402022,Paramount Pictures,3.5/10
6819,Çarsi Pazar,tt4324242,Netflix,no_data
6820,Çok Filim Hareketler Bunlar,tt1640202,Netflix,no_data


In [183]:
df_movie_id_clean_copy.query('title=="10 Things I Hate About You"')	

,title,id,distributor,rating score
13,10 Things I Hate About You,tt0147800,"(Disney+, Netflix)",69%


In [184]:
df_movie_id_clean_copy["distributor"][13]  #

('Disney+', 'Netflix')

In [185]:
pd.concat(g for _, g in df_movie_id_clean_copy.groupby("title") if len(g) > 1) #ci rimangono i film con stesso titolo diversa id (da controllare a mano?)

,title,id,distributor,rating score
114,A Chinese Ghost Story,tt0093978,Netflix,67%
115,A Chinese Ghost Story,tt1861421,no_data,no_data
167,A Perfect Day,tt0469692,Netflix,no_data
168,A Perfect Day,tt3577624,Teodora Film,6.8/10
182,A Second Chance,tt2150139,Netflix,no_data
...,...,...,...,...
6729,Wolves,tt0021558,Woolf & Freedman Film Service,no_data
6730,Wolves,tt1403241,Entertainment One,4.5/10
6731,Wolves,tt4623856,Netflix,4.8/10
6789,"Yours, Mine and Ours",tt0063829,United Artists Corporation,6.5/10


# uniamo tutto

In [217]:
df_movie_id_clean_copy #contiene title, id, distributor, rottenscore,
df_movie_directors #contiene title, director, director gender

,title,director,director gender
0,Sankofa,Haile Gerima,male
1,The Starling,Theodore Melfi,male
2,Confessions of an Invisible Girl,no_data,no_data
3,Intrusion,no_data,no_data
6,Minsara Kanavu,Rajiv Menon,male
...,...,...,...
6775,Dard Divorce,Olaf Ittenbach,male
6776,A Place at the Table,no_data,no_data
6777,10 Things I Hate About You,Gil Junger,male
6781,Harry Brown,Daniel Barber,male


In [216]:
df_movie_directors.head(7)

,title,director,director gender
0,Sankofa,Haile Gerima,male
1,The Starling,Theodore Melfi,male
2,Confessions of an Invisible Girl,no_data,no_data
3,Intrusion,no_data,no_data
6,Minsara Kanavu,Rajiv Menon,male
8,Dark Skies,Scott Stewart,male
9,Paranoia,Robert Luketic,male


In [218]:
match_titles = df_movie_directors['title'].unique()
print(len(match_titles))
match_titles1 = df_movie_id_clean_copy['title'].unique()
print(len(match_titles1))

4727
6677


In [219]:
#wikidata_titles_clean_autors = df_movie_directors["title"].tolist() (già fatto sopra)
len(wikidata_titles_clean_autors) #facciamo una lista dei titoli con autori giusti

3397

In [220]:
index_to_drop =[] #ripuliamo il dataframe di wikidata da tutti i film con autori che non matchano
for  index_n, title in df_movie_id_clean_copy['title'].iteritems():
    if title not in wikidata_titles_clean_autors:

            index_to_drop.append(index_n)
    
df_movie_id_cleaned_drctr = df_movie_id_clean_copy.drop(index_to_drop)
        
        
print(len(df_movie_id_cleaned_drctr))

3297


In [221]:
df_movie_id_cleaned_drctr1 = df_movie_id_cleaned_drctr['title'].unique()
len(df_movie_id_cleaned_drctr1)

3260

In [222]:
print(len(df_movie_id_cleaned_drctr['title']))
print(len(df_movie_directors['title']))

3297
4727


# merge film dataframes


In [223]:
df_movie_id_cleaned_drctr.head()

,title,id,distributor,rating score
1,#Home,tt10534500,Amazon Prime Video,no_data
2,#Roxy,tt5646082,iTunes,no_data
3,#realityhigh,tt6119504,Netflix,40%
4,'76,tt2385126,YouTube,no_data
5,1 Chance 2 Dance,tt2388856,Netflix,no_data


In [224]:
df_movie_directors_copy = df_movie_directors.copy()
df_movie_directors_copy["distributor"] = " "
df_movie_directors_copy["id"] = " "
df_movie_directors_copy["rating score"] = " "
df_movie_directors_copy.head()

,title,director,director gender,distributor,id,rating score
0,Sankofa,Haile Gerima,male,,,
1,The Starling,Theodore Melfi,male,,,
2,Confessions of an Invisible Girl,no_data,no_data,,,
3,Intrusion,no_data,no_data,,,
6,Minsara Kanavu,Rajiv Menon,male,,,


In [240]:
for index_n, title in df_movie_directors_copy['title'].iteritems():
    for index_id, title_id in df_movie_id_cleaned_drctr['title'].iteritems():
        distributor = df_movie_id_cleaned_drctr["distributor"][index_id]
        id = df_movie_id_cleaned_drctr["id"][index_id]
        rating = df_movie_id_cleaned_drctr["rating score"][index_id]
        
        if title == title_id:
            
            df_movie_directors_copy.at[n_index,'distributor']=distributor
            df_movie_directors_copy.at[n_index,'id']=id
            df_movie_directors_copy.at[n_index,'rating score']=rating 
    

In [241]:
df_movie_directors_copy.head(7)

,title,director,director gender,distributor,id,rating score
0,Sankofa,Haile Gerima,male,,,
1,The Starling,Theodore Melfi,male,,,
2,Confessions of an Invisible Girl,no_data,no_data,,,
3,Intrusion,no_data,no_data,,,
6,Minsara Kanavu,Rajiv Menon,male,,,
8,Dark Skies,Scott Stewart,male,,,
9,Paranoia,Robert Luketic,male,,,


# Series

In [14]:
df_serie_title.head(7)

,title,director,gender
0,Ganglands,no_data,no_data
1,Midnight Mass,Mike Flanagan,male
2,The Great British Baking Show,no_data,no_data
3,The Smart Money Woman,no_data,no_data
4,King of Boys: The Return of the King,Kemi Adetiba,female
5,Valeria,Nely Reguera,female
6,I Need Romance,no_data,no_data


In [15]:
df_serie_id.head(7)

,serie,brodcaster,imdbID,rottenscore
0,Ganglands,no_data,no_data,no_data
1,Midnight Mass,Netflix,tt10574558,no_data
2,The Great British Baking Show,BBC One,tt1877368,no_data
3,The Smart Money Woman,Netflix,no_data,no_data
4,King of Boys: The Return of the King,Netflix,no_data,no_data
5,Valeria,Univision,tt1483311,no_data
6,I Need Romance,tvN,tt3340752,no_data
